In [20]:
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import nltk
import re
from nltk.corpus import stopwords

In [2]:
from google.colab import files
upload=files.upload()

Saving train.csv to train.csv


In [3]:
import io

In [4]:
df=pd.read_csv(io.BytesIO(upload['train.csv']))

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
df=df.dropna()

In [14]:
x=df.drop('label',axis=1)
y=df['label']

In [15]:
x.shape

(18285, 4)

In [16]:
y.shape

(18285,)

In [17]:
messages=x.copy()

In [19]:
messages.reset_index(inplace=True)

In [23]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

**Data preprocessing**

In [24]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

In [39]:
from nltk.stem import WordNetLemmatizer
wn=WordNetLemmatizer()
corpus_2=[]

In [38]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [27]:
for i in range(len(messages)):
  review=re.sub('[^A-Za-z]',' ',messages['title'][i])
  review=review.lower()
  review= review.split()
  review=  [ps.stem(word) for word in review if not word in stopwords.words('english')] 
  review=' '.join(review)
  corpus.append(review)

In [40]:
for i in range(len(messages)):
  review=re.sub('[^A-Za-z]',' ',messages['title'][i])
  review=review.lower()
  review= review.split()
  review=  [wn.lemmatize(word) for word in review if not word in stopwords.words('english')] 
  review=' '.join(review)
  corpus_2.append(review)

In [44]:
#vocabulary size and one hot representation
voc_size=5000
one_hot_repr=[one_hot(words,voc_size) for words in corpus]

**Embedding representation**

In [46]:
sent_len=20
embedded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_len)

In [49]:
#creating a model
embeded_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embeded_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [62]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(y)


In [63]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_final,Y_final,test_size=0.33,random_state=42)

In [64]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1289 - accuracy: 0.9478 - val_loss: 0.2244 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0381 - accuracy: 0.9869 - val_loss: 0.3310 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0134 - accuracy: 0.9960 - val_loss: 0.3617 - val_accuracy: 0.9183
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0053 - accuracy: 0.9989 - val_loss: 0.5243 - val_accuracy: 0.9170
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 0.5276 - val_accuracy: 0.9162
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 9.0203e-04 - accuracy: 0.9998 - val_loss: 0.5740 - val_accuracy: 0.9201
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 2.8006e-04 - accuracy: 1.0000 - val_loss: 0.6253 - val_accuracy

In [65]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
y_pred=model.predict_classes(X_test)
confusion=confusion_matrix(Y_test,y_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [66]:
confusion

array([[3148,  271],
       [ 230, 2386]])

In [67]:
accuracy=accuracy_score(Y_test,y_pred)
accuracy

0.9169842584921293